<a href="https://colab.research.google.com/github/jcsh4326/notebook/blob/master/TensorFlow_02_Use_TensorBoard_With_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 在colab中使用TensorBoard
利用ngrok进行内网穿透，访问TensorBoard

详见[stackoverflow](https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab)

In [0]:
#@title import
import numpy as np
import tensorflow as tf

In [0]:
#@title function helper
#@markdown edit layer <br> outputs = inputs * Weights + biases
def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
  # add one more layer and return the output of this layer    
  layer_name = 'layer{}'.format(n_layer)
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='weights')
      tf.summary.histogram('{}/weights'.format(layer_name), Weights)
    with tf.name_scope('biases'):
      biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='biases')
      tf.summary.histogram('{}/biases'.format(layer_name), biases)
    with tf.name_scope('outpus_y'):
      Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
    if activation_function is None:
      outputs = Wx_plus_b
    else:
      outputs = activation_function(Wx_plus_b)
    tf.summary.histogram('{}/outputs'.format(layer_name), outputs)
    return outputs

In [0]:
#@title define placeholder for inputs to network
with tf.name_scope('inputs'):
  # define placeholder for inputs to network
  xs = tf.placeholder(tf.float32, [None, 1], name='x_in')
  ys = tf.placeholder(tf.float32, [None, 1], name='y_in') 

In [0]:
#@title add hidden layer
# add hidden layer
l1 = add_layer(xs, 1, 10, n_layer = 1, activation_function=tf.nn.relu)
# add output layer
prediction = add_layer(l1, 10, 1, n_layer = 2, activation_function=None)

In [0]:
#@title loss function, the error between prediciton and real data
with tf.name_scope('loss'):
  loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), 1))   
  tf.summary.scalar('loss', loss)

In [0]:
#@title train
with tf.name_scope('train'):
  train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [0]:
#@title run
sess = tf.Session()

merged = tf.summary.merge_all()

writer = tf.summary.FileWriter("logs/", sess.graph)

init = tf.global_variables_initializer()
sess.run(init)

In [0]:
#@title train data simulator
x_data = np.linspace(-1, 1, 300, dtype=np.float32)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape).astype(np.float32)
y_data = np.square(x_data) - 0.5 + noise

for i in range(1000):
  sess.run(train_step, feed_dict={xs:x_data, ys:y_data})
  if i % 50 == 0:
    rs = sess.run(merged, feed_dict={xs:x_data, ys:y_data})
    writer.add_summary(rs, i)

In [13]:
#@title install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2018-11-30 06:35:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.173.6.94, 34.196.237.103, 35.173.3.255, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.173.6.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  3.48MB/s    in 1.5s    

2018-11-30 06:35:57 (3.48 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [15]:
#@title ngrok
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://065ce897.ngrok.io
